# Homework 1: Fundamentals of Neural Networks
### Course: CS 5480 (Deep Learning) @ Missouri S&T
### Semester: Spring 2025
### Template Scribe: Dr. Sid Nadendla
## Author: <Type your name here>

In [146]:
# Import Statements
import numpy as np

# sklearn has the MNIST dataset, as well as the supporting functions to preprocess data
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Set seed
np.random.seed(42)

# Problem 1: Primitives

## 1.1 Linear functions

In [147]:
def linear(z,W):   
    # TODO: Implement linear function
    """
    Computes the linear function Y = W * X
    
    Parameters:
    x (numpy.ndarray): K x 1 vector
    W (numpy.ndarray): M x K matrix
    
    Returns:
    numpy.ndarray: M x 1 vector Y
    """
    return np.dot(W, z) 
    
def linear_grad(z,W,mode="default"):
    # TODO: Implement gradient of the linear function -- need to return a different output depending on the mode. 
    # A default mode computes the gradient with respect to the input z, while the weight mode computes the gradient with respect to the weight W.
    M, K = W.shape

    if mode == "default":
        # Gradient with respect to z is W (M x X)
        return W
    elif mode == "weight":
        # Gradient with respect to W is M x M x K tensor
        grad_W = np.zeros(M, M, K)
        for i in range(M):
            grad_W[i, i, :] = z.flatten() # z.flatten(), converts the column vector z(of shape K x 1 into a 1D array of shape K
        return grad_W
    else:
        raise NotImplementedError("Gradient of linear function not implemented")

## 1.2 Activation functions

In [148]:
def relu(x):
    # TODO: Implement ReLU activation
    return np.maximum(0, x)

def relu_grad(y):
    # TODO: Implement the gradient of ReLU activation as a function of output
    """
        Computes the subgradient of the ReLU activation, 
        Returns an M × M dimensional matrix
    """
    """
  
    M = y.shape[0]
    grad = np.zeros((M, M))
    for i in range(M):
        grad[i, i] = 1 if np.all(y[i] > 0) else 0
    return grad
""" 
    #M = y.shape[0]
    #grad = np.diag((y > 0).astype(float))
    #return grad
    return (y > 0).astype(float) 
    
def logistic(x):
    # TODO: Implement logistic activation
    return 1 / (1 + np.exp(-x))

def logistic_grad(y):
    # TODO: Implement the gradient of logistic activation as a function of output
    """
    Computes the gradient of the logistic function.
    Returns an M × M dimensional matrix with negative values.
    """
    return np.diag(y.flatten() * (1 - y.flatten()))

## 1.3 Loss function

In [149]:
def crossentropy(y,y_hat):
    # TODO: Implement forward pass of the loss function
    """
    Parameters:
    y (float): True label (0 or 1)
    y_hat (float): Predicted probability (0 < y_hat < 1)
    
    Returns:
    float: Gradient of binary cross-entropy loss
    """
    if not (y in [0, 1] and 0 < y_hat < 1):
        raise ValueError("Both y must be 0 or 1, and y_hat must be between 0 and 1 (exclusive).")
    
    return - (y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

def crossentropy_grad(y,y_hat):
    
    # TODO: Implement gradient of the loss function
    delta = 1e-6
    if not (np.all(np.isin(y, [0, 1])) and np.all((0 < y_hat) & (y_hat < 1))):
        raise ValueError("Both y must be 0 or 1, and y_hat must be between 0 and 1 (exclusive).")
    
    return (1 - y + delta) / (1 - y_hat + delta) - (y + delta) / (y_hat + delta)



In [150]:
# Run this cell only to test the code written for Problem 1.
def test_problem1():
    """Unit tests for Problem 1 implementations."""
    M, K = 3, 2
    W = np.array([[1, 0], [0, 1], [1, -1]])
    x = np.array([[1], [2]])
    
    # Test linear function
    expected_linear = np.array([[1], [2], [-1]])
    assert np.array_equal(linear(x, W), expected_linear), "Linear function failed"
    
    # Test ReLU function
    expected_relu = np.array([[1], [2], [0]])
    assert np.array_equal(relu(expected_linear), expected_relu), "ReLU function failed"
    
    # Test ReLU gradient
    y = np.array([1, 2, -1])
    expected_output = np.array([1, 1, 0])
    assert np.array_equal(relu_grad(y), expected_output), "1D input test failed"

    # 2D input
    y = np.array([[1, 2], [3, -1]])
    expected_output = np.array([[1, 1], [1, 0]])
    assert np.array_equal(relu_grad(y), expected_output), "2D input test failed"

    
    # Test logistic function
    assert np.allclose(logistic(0), 0.5), "Logistic function failed"
    
    # Test crossentropy function
    assert np.allclose(crossentropy_grad(1, 0.9), (1 - 1) / (1 - 0.9 + 1e-6) - (1 + 1e-6) / (0.9 + 1e-6)), "Crossentropy gradient failed"
    
    print("All tests passed successfully!")
   

test_problem1()


All tests passed successfully!


# Problem 2: NN-2 model

In [151]:
# Abstract class for one layer in NN (DO NOT CHANGE THIS CELL)
class Layer():
    def __init__(self, W, activation):
        self.W  = W
        self.activation = activation

In [152]:
class NN2:
    #def __init__(self):
    def __init__(self, M, K):
        # Two layers in the model architecture
        self.layers = [
            #[Layer(None, relu), 
            #Layer(None, logistic)]
            Layer(None, relu), 
            #Layer(W0, relu),
            #Layer(W1, logistic)]
            Layer(None, logistic)]
    
    def forward(self,x):
        # TODO: Implement the forward pass of the model and return the model output
        if self.layers[0].W is None or self.layers[1].W is None:
            raise ValueError("Weights must be initialized before forward pass.")

        self.z1 = linear(x, self.layers[0].W)
        self.z1_tilde = self.layers[0].activation(self.z1)
        
        self.z2 = linear(self.z1_tilde, self.layers[1].W.T)
        #self.z2 = linear(self.z1_tilde, self.layers[1].W)
        self.y_hat = self.layers[1].activation(self.z2)
        return self.y_hat


    def backward(self,x, y): # made a change here, initially is (self, x)
        # TODO: Implement the backprop algorithm to compute the model gradients
        if self.layers[0].W is None or self.layers[1].W is None:
            raise ValueError("Weights must be initialized before backward pass.")

        N = x.shape[1]
        y_hat = self.forward(x)
        
        dL_dy_hat = crossentropy_grad(y, y_hat)
        dy_hat_dz2 = self.y_hat * (1 - self.y_hat)
        dL_dz2 = dL_dy_hat * dy_hat_dz2

        dL_dw2 = np.dot(self.z1_tilde, dL_dz2.T) / N # np.dot((M, N), (N, 1)) 
        #dL_dw2 = np.dot(self.z1_tilde, dL_dz2) / N # np.dot((M, 1), (1, 1)) 
        dz2_dz1_tilde = self.layers[1].W
        dz1_tilde_dz1 = relu_grad(self.z1)
        dL_dz1 = np.dot(dz2_dz1_tilde, dL_dz2) * dz1_tilde_dz1
        #dL_dz1 = np.dot(dz2_dz1_tilde, dL_dz2.T) * dz1_tilde_dz1
        dL_dW1 = np.dot(dL_dz1, x.T) / N  # x.T, a little bit not sure of why x.T

        return dL_dW1, dL_dw2
        
    def emp_loss_grad(self, train_X, train_y):
        # TODO: Implement the empirical loss gradients for the training set
        return self.backward(train_X, train_y)
        

# Problem 3: Optimization Algorithm - Gradient Descent

In [153]:
# I added a helper function 
def compute_loss(y, y_hat, delta=1e-6):
    """
    Computes the average binary cross-entropy loss over all samples.
    
    Parameters:
    y (numpy.ndarray): True labels with shape (1, N)
    y_hat (numpy.ndarray): Predicted probabilities with shape (1, N)
    delta (float): A small constant to avoid log(0)
    
    Returns:
    float: The average loss over all N samples.
    """
    loss = - (y * np.log(y_hat + delta) + (1 - y) * np.log(1 - y_hat + delta))
    return np.mean(loss)


In [154]:
def gradient_descent(model, train_X, train_y, lr, R):  
    # TODO: Implement gradient descent and return updated weights
    prev_loss = float('inf')
    for r in range(R):
        # Forward pass: get predictions on the full training set
        y_hat = model.forward(train_X)
        # Compute the average cross-entropy loss
        loss = compute_loss(train_y, y_hat)
        print(f"Iteration {r}, Loss: {loss:.4f}")
        
        # Compute gradients via backpropagation
        dL_dW1, dL_dw2 = model.emp_loss_grad(train_X, train_y)
        
        # Update the weights
        model.layers[0].W -= lr * dL_dW1
        model.layers[1].W -= lr * dL_dw2
        
        # Convergence check based on loss change
        if abs(prev_loss - loss) < 1e-6:
            break
        prev_loss = loss
        
    return model   


In [155]:
# Run this cell only to test the code written for Problem 2.
def test_problem2():
    
    
    """Unit tests for gradient descent implementation."""
    M, K, N = 4, 3, 5  # Example sizes
    model = NN2(M, K)  # Pass M and K when initializing

    model.layers[0].W = np.random.randn(M, K)
    model.layers[1].W = np.random.randn(M, 1)
    
    X = np.random.randn(K, N)
    Y = (np.random.rand(1, N) > 0.5).astype(float)  # Binary labels

    model = gradient_descent(model, X, Y, lr=0.01, R=50)
    
    assert model.layers[0].W is not None, "W1 update failed"
    assert model.layers[1].W is not None, "w2 update failed"
    print("Gradient descent test passed!")

# Run the fixed test
test_problem2()


Iteration 0, Loss: 0.4370
Iteration 1, Loss: 0.4367
Iteration 2, Loss: 0.4364
Iteration 3, Loss: 0.4361
Iteration 4, Loss: 0.4358
Iteration 5, Loss: 0.4354
Iteration 6, Loss: 0.4351
Iteration 7, Loss: 0.4348
Iteration 8, Loss: 0.4345
Iteration 9, Loss: 0.4342
Iteration 10, Loss: 0.4338
Iteration 11, Loss: 0.4335
Iteration 12, Loss: 0.4332
Iteration 13, Loss: 0.4329
Iteration 14, Loss: 0.4326
Iteration 15, Loss: 0.4322
Iteration 16, Loss: 0.4319
Iteration 17, Loss: 0.4316
Iteration 18, Loss: 0.4313
Iteration 19, Loss: 0.4309
Iteration 20, Loss: 0.4306
Iteration 21, Loss: 0.4303
Iteration 22, Loss: 0.4300
Iteration 23, Loss: 0.4296
Iteration 24, Loss: 0.4293
Iteration 25, Loss: 0.4290
Iteration 26, Loss: 0.4287
Iteration 27, Loss: 0.4283
Iteration 28, Loss: 0.4280
Iteration 29, Loss: 0.4277
Iteration 30, Loss: 0.4273
Iteration 31, Loss: 0.4270
Iteration 32, Loss: 0.4267
Iteration 33, Loss: 0.4263
Iteration 34, Loss: 0.4260
Iteration 35, Loss: 0.4257
Iteration 36, Loss: 0.4253
Iteration 3

# Problem 4: Training and Testing

## 4.1 Data Loading and Preprocessing

In [156]:
def load_dataset():
    #TODO: Return the data tuple as input (x) and output (y) 
    """Load the MNIST dataset."""
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X, y = mnist.data, mnist.target.astype(int)
    return X, y

def prepare_data(X, y):
    #TODO: Convert output labels in MNIST into binary labels
    """Normalize and preprocess MNIST dataset."""
    # Normalize pixel values (0 to 1)
    X = X / 255.0
    
    # Flatten each image into a 784 x 1 vector and append '-1' as bias term
    X = np.hstack([X, -np.ones((X.shape[0], 1))])  # Now X has shape (70000, 785)
    
    # Convert labels into binary classification: Even digits -> 1, Odd digits -> 0
    y = (y % 2 == 0).astype(int)
    
    return X, y # change X to X.t (785, 70000)
    
def split_train_test(X,y):
    #TODO: return X_train, X_test, y_train, y_test
    
    return train_test_split(X, y, test_size=10000/70000, random_state=42)
   

## 4.2 Model Initialization

In [157]:
# DO NOT CHANGE THIS CELL
def initialize_model(X_train, X_test, y_train, y_test):
    #TODO

    M = 128  # Number of hidden units
    K = X_train.shape[0]  # Input dimension (785 after adding bias)
    W0 = np.random.randn(M, K) * np.sqrt(1.0 / K) # change 0.01 to np.sqrt(1.0 / K )
    W1 = np.random.randn(M, 1) * np.sqrt(1.0 / M)
    

    print(f"Size of W0: {W0.shape}, Size of W1: {W1.shape}")

    
    #two_layer_nn  = NN2()
    two_layer_nn = NN2(M, K)  # Initialize the model
    two_layer_nn.W = [W0, W1]
    two_layer_nn.layers[0].W = W0
    two_layer_nn.layers[1].W = W1

    return two_layer_nn

## 4.3 Model Training

In [158]:
def train_model(model, X_train, y_train):
    #TODO: Use GD to train the model on training dataset
    
    model = gradient_descent(model, X_train, y_train.reshape(1, -1), lr=0.01, R= 100)
    final_W = [model.layers[0].W, model.layers[1].W]  # add this
    return model, final_W # add final_W
    

## 4.4 Model Testing

In [159]:
def test_model(model, X_test, y_test): 
    #TODO: Define accuracy metric and return the accuracy of the trained model on test data
    
    """Evaluate model accuracy on the test dataset."""
    y_pred = model.forward(X_test)  # Forward pass to get predictions
    y_pred_labels = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels
    accuracy = np.mean(y_pred_labels == y_test.reshape(1, -1)) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy
   

##### 5. Run this cell when the entire code-base is ready for testing

In [160]:
#load data
x, y = load_dataset()

#prepare data
x, y = prepare_data(x,y)

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)

X_train = X_train.T  # Now shape (785, n_train), Added
X_test = X_test.T    # Now shape (785, n_test), Added

#initialize model
model = initialize_model(X_train, X_test, y_train, y_test)

#training model
trained_model, final_W = train_model(model, X_train, y_train) # add final_W
#print(f"Completed training model - final W : {final_W}")
print("Layer 0 weights shape:", final_W[0]) #
print("Layer 1 weights shape:", final_W[1]) #

#testing model
accuracy = test_model(model, X_test, y_test) # delete final_W
print(f"Completed testing model - Accuracy : {accuracy}")    

Size of W0: (128, 785), Size of W1: (128, 1)
Iteration 0, Loss: 0.7028
Iteration 1, Loss: 0.7000
Iteration 2, Loss: 0.6973
Iteration 3, Loss: 0.6948
Iteration 4, Loss: 0.6923
Iteration 5, Loss: 0.6899
Iteration 6, Loss: 0.6876
Iteration 7, Loss: 0.6853
Iteration 8, Loss: 0.6831
Iteration 9, Loss: 0.6810
Iteration 10, Loss: 0.6789
Iteration 11, Loss: 0.6769
Iteration 12, Loss: 0.6749
Iteration 13, Loss: 0.6729
Iteration 14, Loss: 0.6710
Iteration 15, Loss: 0.6691
Iteration 16, Loss: 0.6673
Iteration 17, Loss: 0.6655
Iteration 18, Loss: 0.6637
Iteration 19, Loss: 0.6619
Iteration 20, Loss: 0.6601
Iteration 21, Loss: 0.6584
Iteration 22, Loss: 0.6567
Iteration 23, Loss: 0.6550
Iteration 24, Loss: 0.6533
Iteration 25, Loss: 0.6517
Iteration 26, Loss: 0.6500
Iteration 27, Loss: 0.6484
Iteration 28, Loss: 0.6468
Iteration 29, Loss: 0.6452
Iteration 30, Loss: 0.6436
Iteration 31, Loss: 0.6420
Iteration 32, Loss: 0.6404
Iteration 33, Loss: 0.6389
Iteration 34, Loss: 0.6373
Iteration 35, Loss: 

In [161]:
# Bonus try SGD and Adam

In [162]:
# full-batch gradient descent with momentum
def sgd_momentum(model, train_X, train_y, lr=0.01, momentum=0.9, R=100):
    """
    Stochastic Gradient Descent with Momentum optimizer.
    
    Parameters:
      model:  NN2 model.
      train_X: input data of shape (K, N) where K is the number of features and N the number of samples.
      train_y: true labels with shape (1, N).
      lr: learning rate.
      momentum: momentum factor.
      R: number of iterations.
      
    Returns:
      Updated model after training.
    """
    # Initialize velocity for each weight matrix
    v_W1 = np.zeros_like(model.layers[0].W)
    v_W2 = np.zeros_like(model.layers[1].W)
    prev_loss = float('inf')
    
    for r in range(R):
        # Forward pass and compute loss
        y_hat = model.forward(train_X)
        loss = compute_loss(train_y, y_hat)
        #print(f"Iteration {r}, Loss: {loss:.4f}")
        
        # Compute gradients via backpropagation
        dL_dW1, dL_dW2 = model.emp_loss_grad(train_X, train_y)
        
        # Update velocities and then weights
        v_W1 = momentum * v_W1 + lr * dL_dW1
        v_W2 = momentum * v_W2 + lr * dL_dW2
        model.layers[0].W -= v_W1
        model.layers[1].W -= v_W2
        
        if abs(prev_loss - loss) < 1e-6:
            break
        prev_loss = loss
        
    return model



In [163]:
# trained model
trained_model = sgd_momentum(model, X_train, y_train.reshape(1, -1), lr=0.01, momentum=0.9, R=100)


In [164]:
def test_model(model, X_test, y_test):
    """
    Evaluates the model on the test dataset and prints the accuracy.
    
    Parameters:
      model: Your NN2 model after training.
      X_test: Test inputs in shape (K, N) where K is number of features and N is number of test samples.
      y_test: True labels for the test data with shape (1, N) or (N,).
    
    Returns:
      accuracy: The classification accuracy as a percentage.
    """
    # Forward pass on test data
    y_pred = model.forward(X_test)
    
    # Convert predicted probabilities to binary labels using a threshold of 0.5
    y_pred_labels = (y_pred > 0.5).astype(int)
    
    # Ensure y_test is in the correct shape
    y_test = y_test.reshape(1, -1)
    
    # Compute accuracy
    accuracy = np.mean(y_pred_labels == y_test) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    return accuracy


In [165]:
accuracy = test_model(trained_model, X_test, y_test)


Test Accuracy: 87.13%


In [166]:
# Admam Model

In [167]:
def adam(model, train_X, train_y, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, R=100):
    """
    Adam optimizer for training the model.
    
    Parameters:
      model: NN2 model.
      train_X: input data of shape (K, N) where K is the number of features and N is the number of samples.
      train_y: true labels with shape (1, N).
      lr: learning rate.
      beta1: exponential decay rate for the first moment estimates.
      beta2: exponential decay rate for the second moment estimates.
      epsilon: small constant for numerical stability.
      R: number of iterations.
      
    Returns:
      model: the updated model after training.
    """
    # Initialize the first moment (m) and second moment (v) for each weight matrix.
    m_W1 = np.zeros_like(model.layers[0].W)
    v_W1 = np.zeros_like(model.layers[0].W)
    m_W2 = np.zeros_like(model.layers[1].W)
    v_W2 = np.zeros_like(model.layers[1].W)
    
    t = 0
    prev_loss = float('inf')
    
    for r in range(R):
        t += 1
        
        # Forward pass and compute the loss.
        y_hat = model.forward(train_X)
        loss = compute_loss(train_y, y_hat)
        #print(f"Iteration {r}, Loss: {loss:.4f}")
        
        # Compute gradients via backpropagation.
        dL_dW1, dL_dW2 = model.emp_loss_grad(train_X, train_y)
        
        # Update biased first moment estimates.
        m_W1 = beta1 * m_W1 + (1 - beta1) * dL_dW1
        m_W2 = beta1 * m_W2 + (1 - beta1) * dL_dW2
        
        # Update biased second moment estimates.
        v_W1 = beta2 * v_W1 + (1 - beta2) * (dL_dW1 ** 2)
        v_W2 = beta2 * v_W2 + (1 - beta2) * (dL_dW2 ** 2)
        
        # Compute bias-corrected first moment estimates.
        m_W1_hat = m_W1 / (1 - beta1 ** t)
        m_W2_hat = m_W2 / (1 - beta1 ** t)
        
        # Compute bias-corrected second moment estimates.
        v_W1_hat = v_W1 / (1 - beta2 ** t)
        v_W2_hat = v_W2 / (1 - beta2 ** t)
        
        # Update weights.
        model.layers[0].W -= lr * m_W1_hat / (np.sqrt(v_W1_hat) + epsilon)
        model.layers[1].W -= lr * m_W2_hat / (np.sqrt(v_W2_hat) + epsilon)
        
        # Optional: Early stopping if the loss change is small.
        if abs(prev_loss - loss) < 1e-6:
            break
        prev_loss = loss
    
    return model


In [ ]:
# Assume X_train is transposed to shape (785, n_train) and y_train is reshaped to (1, n_train)
trained_model = adam(model, X_train, y_train.reshape(1, -1), lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8, R=100)


In [ ]:
accuracy = test_model(trained_model, X_test, y_test)

In [ ]:
#summarise: The predictions of my model are:  GD is 81.25% , SGD is 87.13%, Adam is 97.95%